## Enterprise Scale ML (ESML) on Azure - AI Factory
- Best Practics Docs about ESML AI Factories:
https://docs.microsoft.com/en-us/azure/cloud-adoption-framework/ready/azure-best-practices/ai-machine-learning-mlops#mlops-at-organizational-scale-ai-factories

![](../azure-enterprise-scale-ml/esml/images/esml-turnkey-small.png)

## ESML AI Factory: Can I just use the Azure ML SDK directly? Instead of the ESML SDK? 
- Yes, You can bypass ESML SDK 100%  (the 4th ingredience) and only take advantage of the 3 other ingredients: part: 1,2,3
    - part 1) `Azure services glued together securely` (ARM / Provisioing / Networking / Infra)
    - part 2) `Azure Devops template, for MLOps` (BUILD and RELEASE pipelines / Networking / Security & Glue) 
    - part 3) `The enterprise datalake design` (ADLS Gen2 storage account, with a folder structure)
    
- That said, part 4) `ESML SDK` provides most `accelerator power`, such as `ESMLPipelinfactory` for Azure ML Pipelines and Azure Datafactory pipelines + some benefits listed below (look at README feature list for all)
    - For an `ML Solution` (ingest,train (mlops),score, writeback) There is `always at least 6 steps you need to create`, step 7 is optional.
    - ESML gives you 5 of these automatically. Not the asterix (*) ones, but the `gold ones` (3,4,5a,5b,6,7)
        - *1) Ingest from source
            - Azure Data Factory: Here ESML provides a DEMO Azure data factory pipeline to show "target". But This is too specific for EMSL. 
                - A COPY Activity from your source (DW/Database) to the IN-folder in ADLS Gen2, Datalake) is needed here.
        - *2) FeatureEngineering "Bronze2Gold"
            - Azure ML Pipeline: ESML PipelineFactory will genereate the pipeline. See `IN_2_GOLD_SCORING` pipeline template (other notebook)
                - *But the content of feature engineering is too specific for ESML. 
                - *And you might need yo create your own Azure ML Pipeline, if the Bronze-Silver-Gold data model does not suit you.
        - `3) TrainModel` (AutoML pipeline - you can also create your own manual Training Azure ML Pipeline)
        - `4) CompareScoringDrift & DataDrift` - Should be promote & Deploy the newly trained model? Needed to refit model to real world changes.
        - `5a) ONLINE: ESMLComputeFactory has a 2-liner: Creates & Deploy AKS Scoring endpoint - Online/Batch (Only up to 5min REST call for BATCHes)`
        - `5b) BATCH: ESMLPipelineFactory has a 2-liner: Creates & Deploys Scoring Azure ML Pipline` - works on the `BRONZE-SILVER-GOLD` data model, or `IN_2_GOLD` datamodel.
            - Azure ML Pipeline: `ESMLPipelineFactory` will genereate the pipeline. See `IN_2_GOLD_SCORING` (other notebook)
        - `6) ConsumeScoring` & `WriteBack` (Azure Data Factory Template pipeline: See `adf` folder: `M11_03a_WRITEBACK_2_DB_Upsert`)
            - The Data factory template `M11_03a_WRITEBACK_2_DB_Upsert` knows where the AML pipline  `ESMLPipelineFactory.IN_2_GOLD_SCORING` scored & written the data.
                - TODO: In the Azure Data factory template: change connection & schema to your Target/DW/Database, and optionally other `ESMLParameters`
                - ESML parameters allows `daily scoring` or other frequences - select your own `date folder`. You can also choose what `model version` the AML pipelone is to score with.
        - `7)ShareBackPipeline` (Write back refined project SILVER data, to MASTER datalake, for others to use
            - ¤¤= This STEP/Pipeline, is of course NOT important if you don't need refined data, reused in your organization.

## - If I only want to REFINE DATA for a Power BI report? 
`Besides the 6-step "ML application" process` above ↑ The ESML SDK gives a data engieer / Data scientists / Power BI ninja, also these benefits, on a `DETAILED level`
- `Datalake aware`: ESML knows the lake structure. You never need to rememeber any paths. Just work with BRONZE, SILVER, GOLD concept + ML Concepts (TRAIN vs INFERENCE)
- `Datasets`: Autoregisters Azure ML Datasets in correct workspace, with naming convention + tags of scoring, split, versioning, and a UI to browse data
    - Datasets does not need to be used for Machine learning. Seet this as a `feature store` for your project. The `model_folder` in the lake, can be `Power BI report datasets`
    - If you just want to `REFINE data for a Power BI report`, you can leverage the same BRONZE,SILVER, GOLD concept and the `AutoLake`
- `Deploy application/code` across 3 environments/3 subscriptions: Dev,Test, Prod
    - 1-liner DEPLOY a model to online AKS webservice in DEV or to TEST or PROD, but anything can be deployed...
    - What we deploy, can be a WebApplication, does not need to be a ML-model in the AKS Webservice.
- `Governance`: ENTERPRISE SPECIFIC settings, global for all projects, and `PROJECT SPECIFIC` settings
    - `DEV, TEST, PROD SETTINGS`: Settings for: Performance & Compute (Train, Inference), Training time, 
    - `DEV,TEST,PROD PROJECTS `: A project has a `set of Azure PaaS services` that can talk, due to ESML SDK glue:
        - `Azure Databricks` -> (can talk & read/write) to the `Datalake`, due to the ESML mount/mappings & built in security/Networking
        - `Azure Databricks` -> (can talk & read/write) to `Azure ML Studio` (and vice versa) due to the ESML settings & built security & built in security/Networking
        - `Azure Datafactory` -> (can talk) to `Datalake` and `Azure Databricks` and `Azure ML Studio` 
            - Due to ESML built in security/Networking (also bootstrap piplines for `WriteBackToMaster`)
        - `Azure Devops` can be used due to security/Networking
        - `Dev->Test->Prod`: `DEV` services can only talk to DEV (Networking/Security), and the neighbour TEST, but never jump `DEV` to talk or deploy directly to `PROD` services
- `Security`: Networking & Identity & Security (ESML SDK knows how to speak with vNets and Private link, and Keyvaults)
    - `Dev->Test->Prod`: `DEV` services can only talk to DEV (Networking/Security), and the neighbour TEST, but never jump `DEV` to talk or deploy directly to `PROD` services
    - `Private Link (Azure backbone)` is the default EMSL networking setup, when services talk to each other.
        - Exceptions of private link: Sometimes Azure DAtabricks is only vNet injected. (Azure Devops build agent is on same vNet only)
-  (Plus the ML parts in detail)
    - `ML`: Test_Set scoring with a 1-liner, registers this as TAGS on GOLD_TEST set in Azure ML Studio, and TAGS on best MODEL at run. (`once and only once` to calcuate scoring)
    - `Azure ML pipeline: Train`: AutoML training with a 2-liner
    - `Azure ML pipeline: Score`: IN_2_GOLD_SCOREING pipeline with a 2-liner
    - `MLOps pipline`: All 6 steps `for FREE` when using ESML, including `SCORING DRIFT` and Dev,Test,Prod aware when comparing `SCORING DRIFT`
    - `MLOps:Scoring drift` across 3 environments, a 1-liner, `promotes` the model to correct ENVIRONMENT (dev,test, prod) if `better` (gets promoted)

    

![](../azure-enterprise-scale-ml/esml/images/aml-pipeline_batch_ppt-3.png)

# ...So...enough info....lets START CONFIGURE! 

# Howto - configure `ESML Python SDK` for a MODEL? 
- ESML comes with 2 DEMO models: 1 Classification(Titanic) and 1 Regression (Diabetes) `TASK_TYPE` 
    - DATA: You have the TITANIC and DIABETES MODEL folders with data in ESML GIT, or in your local folder: `\azure-enterprise-scale-ml\copy_my_subfolders_to_my_grandparent\demo_data\`
    - 1) First, we need to configure the DATA you want to use for MODEL, that is the ESML AutoLake
    - `*`2) Then, optinally adjust our definition of SCORING_DRIFT - scoring `WEIGHTS` on `regression` or `classification` TASK_TYPE

`*Optional`

1) CONFIGURE MODEL & DATA - `LAKE_SETTINGS`
- Mandatory

![](../azure-enterprise-scale-ml/esml/images/01_setup_model_1.png)

## LAKE_SETTINGS - More info
- 1) lake_settings.json: Point at the "model_folder" and filter your "dataset_folders"


## 1) lake_settings
- Here you can see an easy way to have multiple models, and that `the "_" prefix has a model "inactive"`
- 
- 
![](../azure-enterprise-scale-ml/esml/images/01_setup_model_2.png)

## 2) OPTIONAL - model_settings "Whats the BEST model, according to YOU?"
- Purpose: For SCORING-DRIFT to know what metrics to use when COMPARING `compare_metrics` that YOU control, and can put `WEIGHTs` on also.
- See also the `"docs1"`, `"docs2"`,`"docs3"` text in image
- All else, you can set to 0.0 to have no `WEIGHTS` when comparing scoring for model A and B, to see if we want ot promote model A
- ![](../azure-enterprise-scale-ml/esml/images/01_setup_model_3.png)


## 3) DATA: CONFIGURE `Date_Folder for DATA to use` (demo data is already configured)
- Per `ENVIRONMENT (dev,test,prod)` and per `TRAINING` and `INFERENCE` you can have different data, you can also `choose what MODEL_VERSION to score with`, at INFERENCE
- These .JSON files can be overridden by ESMLProject Constructor, and/or by putting these file in the ESML AutoLake's `active` folder
- From `Azure Datafactory`  you use regular Azure ML `PipelineParameter`'s (scoring_folder_date, modelv_verion) that is sent to the  `ESMLPipelinefactory` genereated Azure ML Pipeline
    - SCORING & WriteBack: See ESMLPipelineFactory `ESMLPipelineFactory.IN_2_GOLD_SCORING` and notebook:  `esml_classification_4_esml_batch_pipeline.ipynb` or `esml_regression_7_b_esml_batch_pipeline.ipynb`

- ![](../azure-enterprise-scale-ml/esml/images/01_setup_model_4.png)
- ![](../azure-enterprise-scale-ml/esml/images/01_setup_model_5.png)

## HIGHLY Optional: Play around different ways to "Override" ESMLProject ACTIVE settings
- You can bypass ESML SDK 100% - the 4th ingredience  - and only take advantage of the 3 other ingrediences: 1,2,3
    - 1) `Azure services glued together securely` (ARM / Provisioing / Networking / Infra)
    - 2) `Azure Devops template, for MLOps` (BUILD and RELEASE pipelines / Networking / Security & Glue) 
    - 3) `The enterprise datalake design` (ADLS Gen2 storage account, with a folder structure)
    

## Howto override above JSON-files - "Active" `date_folder` to TRAIN or INFERENCE and `inference_model_version`? 
- A) `CONSTRUCTOR / Python commandline override:` These .JSON files can be overridden by ESMLProject Constructor
    - `p = ESMLProject(param_esml_env,param_inference_model_version,param_scoring_folder_date,param_train_in_folder_date)`
    - ESML will then overwrite LOCAL json-files, and never look into the DataLake `active` folders
    - `Useful scenario:` Send as `ArgParse` from `Azure Devops - MLOps pipeline` after `code_change`, to set the ESML evnironment (dev,test,prod)
        - MLOps `New Code` trigger
        - Note: This is excately what the ESML MLOps template does: `p = ESMLProject.get_project_from_env_command_line()`
- B) `LAKE override:` You can also put these .JSON files in the ESML AutoLake's `active` folder 
    - EMSL will 1st read from lake, and overwrite LOCAL .json files (See images `For TRAINING` and `For INFERENCE` below)
    - `Useful scenario:` Write these files with `Azure Datafactory` after ingestion pipeline
        - MLOps `New data` trigger



### B) Override via LAKE files

## For TRAINING (in_folder to train on)
- ![](../azure-enterprise-scale-ml/esml/images/01_setup_model_7.png)

## For INFERENCE (model_version, date_folder_to_score)
- Here is the "active/most recent" per model_version seen - but most often overridden by parameters sent to the Azure ML Pipeline, that ESMLPipelineFactory generated.
- ![](../azure-enterprise-scale-ml/esml/images/01_setup_model_8.png)

# TROUBLE SHOOTING!

## Q: p.init()  gives error, cannot find Dataset files in datalake? 
 - `StreamAccessException was caused by NotFoundException.`
## A: 
- The usual case is that the "Date_Folder" (DateTime) in active/active_in_folder.json settings is wrong (they dont point at correct data_folder in datalake)
- Another thing, can be that you have `p.inference_mode=True` is you want to TRAIN model....it should be False when training, for it to look in the `TRAIN lake-folder structure`

....Look at this path with Azure Data explorer, to see that you have data...that mathes the PATH in the Error message....yes, there is a path there somewhere, along with all the text : )


- ![](../azure-enterprise-scale-ml/esml/images/01_setup_model_6.png)